# LlamaIndex と AgentCore Memory - 法務文書アナライザー（短期メモリ）

## はじめに

このノートブックでは、Amazon Bedrock AgentCore Memory 機能を LlamaIndex と統合して法務文書アナライザーを作成する方法を紹介します。単一の法務分析セッション内での**短期メモリ**の永続化に焦点を当て、法務レビュー全体を通じてアナライザーが契約条項、判例、コンプライアンス問題を記憶できるようにします。

## アーキテクチャの概要

![LlamaIndex AgentCore 短期メモリアーキテクチャ](LlamaIndex-AgentCore-STM-Arch.png)

## チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントのユースケース | 法務文書アナライザー                                                            |
| エージェントフレームワーク | LlamaIndex                                                                      |
| LLM モデル          | Anthropic Claude 3.7 Sonnet                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、LlamaIndex エージェント、法務分析ツール                    |
| 難易度              | 初級                                                                            |

学習内容：
- 法務文書分析用の AgentCore Memory の作成
- 法務ワークフロー向け LlamaIndex ネイティブメモリ統合の使用
- 契約分析のための法務専用ツールの構築
- 単一の分析セッション内での法務コンテキストの維持
- メモリ境界とセッション分離のテスト

## シナリオのコンテキスト

この例では、弁護士が単一の法務レビューセッション内で契約の分析、法的問題の追跡、コンプライアンス要件の管理を支援する「法務文書アナライザー」を作成します。アナライザーは AgentCore Memory を使用して、分析全体を通じて契約条項、リスク評価、判例、コンプライアンス問題に関するコンテキストを維持します。

## 前提条件

- Python 3.10以上
- 適切な権限を持つ AWS アカウント
- AgentCore Memory 権限を持つ AWS IAM ロール：
  - `bedrock-agentcore:CreateMemory`
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents`
  - `bedrock-agentcore:RetrieveMemories`
- Amazon Bedrock モデルへのアクセス

## ステップ 1: 依存関係のインストールとセットアップ

In [ ]:
# Install necessary libraries
%pip install llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse boto3

In [ ]:
# Import required components
from bedrock_agentcore.memory import MemoryClient
from llama_index.memory.bedrock_agentcore import AgentCoreMemory, AgentCoreMemoryContext
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
from datetime import datetime
import os

## ステップ 2: AgentCore Memory の設定

法務アナライザー用の AgentCore Memory リソースを作成または取得します：

In [ ]:
# Create AgentCore Memory resource
region = os.getenv('AWS_REGION', 'us-east-1')
client = MemoryClient(region_name=region)

try:
    response = client.create_memory_and_wait(
        name=f'LegalAnalyzerShortTerm_{int(datetime.now().timestamp())}',
        description='Legal document analyzer short-term memory for single session context',
        strategies=[],
        event_expiry_days=7,
        max_wait=300,
        poll_interval=10
    )
    memory_id = response['id']
    print(f"✅ Created AgentCore Memory: {memory_id}")
except Exception as e:
    print(f"❌ Error creating memory: {e}")
    memory_id = "your-memory-id-here"  # Replace with existing memory ID

## ステップ 3: 法務分析ツールの実装

法務文書分析用の専門ツールを定義します：

In [ ]:
def analyze_contract_clause(clause_text: str, clause_type: str, risk_level: str) -> str:
    """Analyze a contract clause and assess its risk level"""
    print(f"⚖️ Analyzed {clause_type} clause (Risk: {risk_level})")
    return f"Analyzed {clause_type} clause with {risk_level} risk assessment"

def track_legal_issue(issue: str, priority: str, status: str) -> str:
    """Track legal issue with priority and status"""
    print(f"📋 Tracking legal issue: {issue} ({priority} priority, {status})")
    return f"Now tracking legal issue: {issue}"

def save_legal_precedent(case_name: str, jurisdiction: str, relevance: str) -> str:
    """Save legal precedent with jurisdiction and relevance"""
    print(f"📚 Saved legal precedent: {case_name} ({jurisdiction})")
    return f"Saved legal precedent: {case_name}"

def flag_compliance_issue(regulation: str, violation_type: str, severity: str) -> str:
    """Flag compliance issue with regulation and severity"""
    print(f"🚨 Flagged {severity} compliance issue: {regulation}")
    return f"Flagged compliance issue: {regulation}"

# Create tool objects for the agent
legal_tools = [
    FunctionTool.from_defaults(fn=analyze_contract_clause),
    FunctionTool.from_defaults(fn=track_legal_issue),
    FunctionTool.from_defaults(fn=save_legal_precedent),
    FunctionTool.from_defaults(fn=flag_compliance_issue)
]

## ステップ 4: LlamaIndex エージェントの実装

短期メモリコンテキストを持つ法務アナライザーエージェントを作成します：

In [ ]:
# Configuration for SHORT-TERM memory (single session)
MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Create memory context for single session
context = AgentCoreMemoryContext(
    actor_id="legal-analyst",
    memory_id=memory_id,
    session_id="legal-analysis-session-today",  # Same session throughout
    namespace="/legal-analysis"
)

# Initialize AgentCore Memory and LLM
agentcore_memory = AgentCoreMemory(context=context)
llm = BedrockConverse(model=MODEL_ID)

# Create the legal analyzer agent
legal_agent = FunctionAgent(
    tools=legal_tools,
    llm=llm,
    verbose=True
)

print("✅ Legal Document Analyzer with short-term memory is ready!")

## ステップ 5: 短期メモリ機能のテスト

包括的な契約分析セッションを通じて法務アナライザーの短期メモリをテストしましょう。

### テスト 1: ケースのセットアップと初期化

In [ ]:
# Initialize legal analysis session with detailed context
response = await legal_agent.run(
    "I'm Attorney Maria Johnson from Johnson & Associates, analyzing a $5M software licensing agreement "
    "between TechCorp (licensor) and DataSoft Inc (licensee). Track this as 'Software License Review' "
    "with critical priority and active status. Contract value: $5M over 3 years.",
    memory=agentcore_memory
)

print("🎯 Case Setup:")
print(response)

### テスト 2: 契約条項の分析

In [ ]:
# Analyze liability clause with specific terms
response = await legal_agent.run(
    "Analyze this liability clause: 'TechCorp's total liability shall not exceed $50,000 for any direct damages "
    "and excludes all indirect, consequential, or punitive damages.' This is a 'Liability Limitation' clause "
    "with 'High' risk level due to low cap vs contract value.",
    memory=agentcore_memory
)

print("⚖️ Liability Clause Analysis:")
print(response)

In [ ]:
# Analyze termination clause with notice periods
response = await legal_agent.run(
    "Analyze termination clause: 'Either party may terminate with 90 days written notice. "
    "TechCorp may terminate immediately for material breach or non-payment exceeding 30 days.' "
    "This is a 'Termination' clause with 'Medium' risk level.",
    memory=agentcore_memory
)

print("📋 Termination Clause Analysis:")
print(response)

### テスト 3: 契約コンテキストの想起

In [ ]:
# Test contract context and risk assessment recall
response = await legal_agent.run(
    "What contract am I analyzing? Who are the parties, what's the value, and what's my assessment of the liability cap?",
    memory=agentcore_memory
)

print("🧠 Contract Context Recall:")
print(response)
print("\n✅ Expected: TechCorp/DataSoft, $5M contract, $50K liability cap (high risk)")

### テスト 4: 詳細条項の想起

In [ ]:
# Test specific clause details recall
response = await legal_agent.run(
    "What are the exact termination notice periods I found? What triggers immediate termination?",
    memory=agentcore_memory
)

print("📋 Termination Details Recall:")
print(response)
print("\n✅ Expected: 90 days notice, immediate for breach or 30+ day non-payment")

### テスト 5: 法的判例の統合

In [ ]:
# Save relevant precedent with case details
response = await legal_agent.run(
    "Save legal precedent: 'TechSoft Inc. v. MegaCorp' from 'Delaware Superior Court' with 'Critical' relevance. "
    "This case established that liability caps below 1% of contract value are unconscionable in software licensing.",
    memory=agentcore_memory
)

print("📚 Legal Precedent Saved:")
print(response)

### テスト 6: リスク評価の推論

In [ ]:
# Test risk assessment reasoning
response = await legal_agent.run(
    "Why did I assess the liability clause as high risk? What's the mathematical relationship between the cap and contract value?",
    memory=agentcore_memory
)

print("🤔 Risk Assessment Reasoning:")
print(response)
print("\n✅ Expected: $50K cap vs $5M contract = 1% ratio, high risk due to low percentage")

### テスト 7: コンプライアンス問題のフラグ付け

In [ ]:
# Flag compliance issue with regulatory details
response = await legal_agent.run(
    "Flag compliance issue: 'GDPR Article 82 Data Protection' violation type 'Inadequate Liability Coverage' "
    "with 'Critical' severity. The $50K cap is insufficient for potential GDPR fines up to 4% of annual revenue.",
    memory=agentcore_memory
)

print("🚨 GDPR Compliance Issue:")
print(response)

### テスト 8: 判例の適用

In [ ]:
# Test precedent application to current case
response = await legal_agent.run(
    "How does the TechSoft v. MegaCorp precedent apply to my current contract analysis? "
    "What does it suggest about the liability clause?",
    memory=agentcore_memory
)

print("⚖️ Precedent Application:")
print(response)
print("\n✅ Expected: Both have ~1% liability caps, precedent suggests unconscionability")

### テスト 9: 包括的なリスク評価

In [ ]:
# Comprehensive risk assessment query
response = await legal_agent.run(
    "Provide a comprehensive risk assessment for DataSoft: What are all the risks I've identified, "
    "their severity levels, and supporting precedents?",
    memory=agentcore_memory
)

print("📊 Comprehensive Risk Assessment:")
print(response)
print("\n✅ Expected: High risk liability cap, GDPR compliance issues, TechSoft precedent support")

## ステップ 6: セッション境界のテスト

異なるセッションを作成して短期メモリの境界をテストしましょう：

In [ ]:
# Create a different session context
new_session_context = AgentCoreMemoryContext(
    actor_id="legal-analyst",
    memory_id=memory_id,
    session_id="different-legal-session",  # Different session ID
    namespace="/legal-analysis"
)

new_session_memory = AgentCoreMemory(context=new_session_context)

# Test memory isolation
response = await legal_agent.run(
    "What contracts am I analyzing? What liability caps and compliance issues have I found?",
    memory=new_session_memory
)

print("🚧 Session Boundary Test (Different Session):")
print(response)
print("\n✅ Expected: Limited or no recall from previous session (short-term memory boundary)")

In [ ]:
# Return to original session to verify persistence
response = await legal_agent.run(
    "Back in my original session - what was the exact liability cap amount and GDPR compliance issue I identified?",
    memory=agentcore_memory  # Original session memory
)

print("🔄 Original Session Return:")
print(response)
print("\n✅ Expected: Full recall of $50K cap, GDPR Article 82 issue")

## 自動テスト検証
これらのセルを実行して、メモリ統合が正しく機能していることを検証します：

In [ ]:
# Define validation functions inline
class TestValidator:
    def __init__(self):
        self.results = {}
    
    def validate_memory_recall(self, response):
        """Check if agent can recall information from earlier in the session"""
        # Check for substantive response (not just "I don't know")
        has_content = len(response) > 50
        # Check for memory indicators
        has_memory_indicators = any(word in response.lower() for word in 
            ['earlier', 'mentioned', 'discussed', 'previously', 'you', 'we', 'our'])
        return "✅ PASS" if (has_content and has_memory_indicators) else "❌ FAIL"
    
    def validate_session_memory(self, response):
        """Check if agent maintains context within session"""
        has_memory_content = len(response) > 100 and any(word in response.lower() for word in 
            ['previous', 'earlier', 'mentioned', 'discussed', 'before', 'already'])
        return "✅ PASS" if has_memory_content else "❌ FAIL"
    
    def validate_cross_reference(self, response):
        """Check if agent can connect current query to previous context"""
        # Look for connecting language
        connecting_words = ['relate', 'connection', 'previous', 'earlier', 'discussed', 
                           'mentioned', 'context', 'based on', 'as we', 'as i']
        has_connection = any(word in response.lower() for word in connecting_words)
        has_substance = len(response) > 80
        return "✅ PASS" if (has_connection and has_substance) else "❌ FAIL"
    
    def run_validation_summary(self, test_results):
        print("🧪 COMPREHENSIVE TEST VALIDATION SUMMARY")
        print("=" * 60)
        
        total_tests = len(test_results)
        passed_tests = sum(1 for result in test_results.values() if "PASS" in result)
        pass_rate = (passed_tests / total_tests * 100) if total_tests > 0 else 0
        
        for test_name, result in test_results.items():
            print(f"{test_name}: {result}")
        
        print("=" * 60)
        print(f"📊 Overall Pass Rate: {passed_tests}/{total_tests} ({pass_rate:.1f}%)")
        
        if pass_rate >= 80:
            print("✅ EXCELLENT: Memory integration working correctly!")
        elif pass_rate >= 60:
            print("⚠️  GOOD: Most memory features working, some issues to investigate")
        else:
            print("❌ NEEDS ATTENTION: Memory integration has significant issues")
        
        return pass_rate

validator = TestValidator()
print("✅ Validation functions loaded!")

In [ ]:
# Run all validation tests
test_results = {}

# Test 1: Memory recall - can the agent recall what was discussed?
response1 = await legal_agent.run("What have we discussed so far in this session?", memory=agentcore_memory)
test_results['Memory Recall'] = validator.validate_memory_recall(str(response1))
print(f"レスポンス 1 の長さ: {len(str(response1))} 文字")

# Test 2: Session memory - does the agent maintain context?
response2 = await legal_agent.run("What did we talk about earlier?", memory=agentcore_memory)
test_results['Session Memory'] = validator.validate_session_memory(str(response2))
print(f"レスポンス 2 の長さ: {len(str(response2))} 文字")

# Test 3: Cross-reference capability - can it connect to previous context?
response3 = await legal_agent.run("How does this relate to what we discussed before?", memory=agentcore_memory)
test_results['Cross Reference'] = validator.validate_cross_reference(str(response3))
print(f"レスポンス 3 の長さ: {len(str(response3))} 文字")

# Display results
validator.run_validation_summary(test_results)

## まとめ

このノートブックでは、以下を実演しました：

- **短期メモリ統合**: セッションスコープの法務分析のために AgentCore Memory を LlamaIndex と使用

- **法務専用ツール**: 契約条項分析、判例管理、コンプライアンス追跡

- **文脈的な法務分析**: 契約の詳細、リスク評価、判例をアナライザーが記憶

- **リスク評価の推論**: 責任上限を契約価値や法的判例と接続

- **セッション境界**: 異なる法務分析セッション間でのメモリ分離

- **コンプライアンス管理**: 規制問題とその重大度レベルの追跡

法務文書アナライザーは、短期メモリが単一のセッション内で包括的な契約分析を可能にしながら、異なる法務案件間で明確な境界を維持する方法を示しています。

## クリーンアップ

このノートブックで使用したリソースをクリーンアップするためにメモリを削除しましょう：

In [ ]:
# Clean up AgentCore Memory resource
try:
    client.delete_memory(memory_id)
    print(f"✅ Successfully deleted memory: {memory_id}")
except Exception as e:
    print(f"❌ Error deleting memory: {e}")